# Pipeline for a data warehouse
### Data Engineering Capstone Project

#### Project Summary
The project combines data of the hackernews feed with formula one race dates of a given year

The project follows the follow steps:
* _Step 1_: Scope the Project and Gather Data

* _Step 2_: Explore and Assess the Data

* _Step 3_: Define the Data Model

* _Step 4_: Run ETL to Model the Data

* _Step 5_: Complete Project Write Up

In [63]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, desc, asc
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

### Step 1: Scope the Project and Gather Data

#### Scope 
We will use the provided datasets by udacity, to create a source-of-truth database that can be used for queries and as data source for BI apps.
Provided datasets: 
The main dataset includes data about _Immigration in the United States of America_ from [here](https://www.trade.gov/national-travel-and-tourism-office).
Supplementary datasets provided are:
- U.S. city _demographics_ from [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
- _Temperature_ data from [here](https://www.trade.gov/national-travel-and-tourism-office)
- Data on _airport codes_ from [here](https://datahub.io/core/airport-codes#data)

#### Describe and Gather Data 
The exploration of the used datasets can be found in `data/exploration.ipynb`.

In [64]:
spark = SparkSession \
    .builder \
    .appName("Capstone Project") \
    .getOrCreate()

In [65]:
sas_data_part = '/Users/joebsbar/Documents/GitBarbara/data-engineering-nd/capstone-project/data/sas_data/part-00000-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet'
df_immigration = spark.read.parquet(sas_data_part)
df_immigration.count()
df_immigration.printSchema()
df_immigration.head()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

Row(cicid=6.0, i94yr=2016.0, i94mon=4.0, i94cit=692.0, i94res=692.0, i94port='XXX', arrdate=20573.0, i94mode=None, i94addr=None, depdate=None, i94bir=37.0, i94visa=2.0, count=1.0, dtadfile=None, visapost=None, occup=None, entdepa='T', entdepd=None, entdepu='U', matflag=None, biryear=1979.0, dtaddto='10282016', gender=None, insnum=None, airline=None, admnum=1897628485.0, fltno=None, visatype='B2')

In [66]:
df_immigration.describe("biryear").show()

+-------+------------------+
|summary|           biryear|
+-------+------------------+
|  count|            219268|
|   mean|1976.1463277815276|
| stddev|17.441783460475794|
|    min|            1916.0|
|    max|            2016.0|
+-------+------------------+



In [68]:
df_immigration.dropDuplicates(['cicid']).sort("gender").show()

+-------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|  cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|
+-------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|13811.0|2016.0|   4.0| 117.0| 117.0|    ATL|20545.0|    1.0|     IL|20549.0|  61.0|    2.0|  1.0|20160401|    null| null|      O|      O|   null|      M| 1955.0|06292016|  null|  null|     DL|5.5448210733E10|00065|      WT|
|49895.0|2016.0|   4.0| 148.0| 112.0|    CHI|20545.0|    1.0|     IL|20549.0|  44.0|    2.0|  1.0|20

In [91]:
from pyspark.sql.types import DateType

#df_immigration.createOrReplaceTempView("immigration")

# add timestamp arrival_date
#transform and add new column
# https://online.stat.psu.edu/stat481/book/export/html/702
# https://stackoverflow.com/questions/36412864/convert-numeric-sas-date-to-datetime-in-pandas
# only immigrations if us

@udf
def sas_to_timestamp(date_sas: float) -> int:
    """
    params: data_sas: a date in sas format
    return: a timestamp in seconds
    """

    if date_sas:
        datetime = pd.to_timedelta((date_sas), unit='D') + pd.Timestamp('1960-1-1')
        timestamp = datetime.timestamp()
        return timestamp
        
df_immigration = df_immigration.withColumn('arrival_ts', sas_to_timestamp(df_immigration['arrdate']))
df_immigration = df_immigration.withColumn('departure_ts', sas_to_timestamp(df_immigration['depdate']))
df_immigration.show(1)

// Todo create fact table
fact_immigration = """

"""



+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+----------+------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|       admnum|fltno|visatype|arrival_ts|departure_ts|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+-------------+-----+--------+----------+------------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null|1.897628485E9| null|      B2|1.461888E9|        null|
+-----+------+------

1461888000.0
1459987200.0
1459468800.0
1472083200.0
1459468800.0
1461369600.0
1459468800.0
1461369600.0
1459468800.0
1460332800.0
1459468800.0
1460592000.0
1459468800.0
1460592000.0
1459468800.0
1460160000.0
1459468800.0
1460937600.0
1459468800.0
1470355200.0
1459468800.0
1460246400.0
1459468800.0
1459814400.0
1459468800.0
1459814400.0
1459468800.0
1460851200.0
1459468800.0
1462320000.0
1459468800.0
1465171200.0
1459468800.0
1460246400.0
1459468800.0
1459468800.0
1459468800.0
1460851200.0
1459468800.0
1461369600.0
1459468800.0
1462060800.0
1459468800.0
1461974400.0
1459468800.0
1460246400.0
1459468800.0
1460937600.0
1459468800.0
1462492800.0
1459468800.0
1469836800.0
1459468800.0
1461801600.0
1459468800.0
1461801600.0
1459468800.0
1461024000.0
1459468800.0
1460332800.0
1459468800.0
1460592000.0
1459468800.0
1459555200.0
1459468800.0
1460332800.0
1459468800.0
1459814400.0
1459468800.0
1459814400.0
1459468800.0
1459555200.0
1459468800.0
1459468800.0
1462492800.0
1459468800.0
1460764800.0

## Temperature Data

In [ ]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = pd.read_csv(fname)
df_temperature.head()

## US City Demographics

In [ ]:
fname = 'us-cities-demographics.csv'
df_temperature = pd.read_csv(fname)
df_temperature.head()

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
The data exploration can be found in `data/exploration.ipynb`

#### Cleaning Steps

#### Cleaning Steps - Immigration Data





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.